In [149]:
from selenium import webdriver
import requests
import time
import re
from bs4 import BeautifulSoup
import pickle

In [ ]:
# selenium version
def parse_content():
    '''글에 있는 내용을 스크래핑하는 함수
    
    output: title, date, question, answer, office를 저장한 딕셔너리'''
    
    return {"title":driver.find_elements_by_css_selector("#Form th.tit")[0].text,
            "date":driver.find_elements_by_css_selector("#Form .date")[0].text,
            "question":driver.find_elements_by_css_selector(".questionDiv tbody div")[0].text,
            "answer":driver.find_elements_by_css_selector(".answerTxt")[0].text,
            "office":driver.find_elements_by_css_selector(".civilPart .pl10")[0].text}

In [ ]:
# selenium version
if __name__ == "__main__":
    driver = webdriver.Chrome()
    driver.get("https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid")

    contents = []
    postList = []

    i = 1

    try:
        while True:
            # 해당 페이지로 이동하는 자바스크립트 함수를 실행한다.
            driver.execute_script("javascript:gotoPage({})".format(i))

            # 현재 페이지에서 볼 수 있는 글 목록을 모두 리스트에 저장한다.
            temp = [_.get_attribute("href") for _ in driver.find_elements_by_css_selector(".taL a")]
            postList.extend(temp)

            while postList:
                # 내용으로 이동한다
                driver.execute_script(postList.pop(0))

                # 제목, 날짜, 질문내용, 답변내용, 담당부서를 딕셔너리형태로 리스트에 추가한다.
                contents.append(parse_content())
                # 글 목록으로 돌아간다.
                driver.back()

            # 다음 페이지로 넘어가도록 i를 증가시킨다.
            i += 1
            if i % 100 == 0:
                print(i)
    except:
        print("crawling finished")

In [ ]:
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    '''
    request 패키지를 이용해서 error handling
    '''
    try:
        resp = requests.request(method, url,params=param, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(maxretries)
            time.sleep(timeout)
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [145]:
# BeautifulSoup version
def get_href():
    ''' 국민신문고 민원 목록 페이지에서 글로 가는 링크를 찾아오는 함수
    
    input:
        
    
    output:
        links: 글로 가는 링크(href)를 담은 리스트 
    '''
    
    page_url = "https://www.epeople.go.kr/jsp/user/pc/policy/uPcOpenCivilList.paid"
    page_param = {"pageNum":1}
    links = []
    
    try:
        while True:
            # 민원 목록 페이지를 html로 parsing
            html = download("get", page_url, page_param)
            dom = BeautifulSoup(html.text)

            # 글로 가는 링크를 리스트에 넣는다.
            links.extend([_["href"] for _ in dom.select("#pForm .taL a")])

            # 1000 page마다 표시
            if page_param["pageNum"] % 1000 == 0:
                print("page: ", page_param["pageNum"])

            # 다음 페이지로 넘어간다.
            page_param["pageNum"] += 1
            
    except:
        print("finished")
        
    return links

In [144]:
# BeautifulSoup version
def parse_content(url, params):
    ''' url과 params를 받아서 해당 글에 있는 민원 정보를 받아오는 함수
    
    input:
        url:
        params:
        
    output:
        딕셔너리: title, date, question, answer, office가 저장돼 있다. 
    '''
    
    # 해당 글에 있는 html을 parsing
    html = download("get", url, params)
    dom = BeautifulSoup(html.text)
    
    return {"title":dom.select_one("#Form th.tit").text,
            "date":dom.select_one("#Form .date").text,
            "question":dom.select_one(".questionDiv tbody div").text,
            "answer":dom.select_one(".answerTxt").text,
            "office":dom.select_one(".civilPart .pl10").text}

100

In [ ]:
# 글 link 목록 생성
hrefs = get_href()

contentURL = "https://www.epeople.go.kr/jsp/user/pc/policy/UPcUnionPolicyDetail.paid"
params = {
    "sdetail":"1",
    "strFlag":"",
    "strArea":"",
    "strBody":"",
    "cat_name":"",
    "strFrom_ex":"",
    "strTo_ex":"",
    "pageNum":"",
    "s_anc_c":"",
    "flag":"3",
    "faq_no_n":None,
    "civil_no_c":"",
    "peti_no_c":"",
    "strFrom":"",
    "strTo":"",
    "so_gubun1":"",
    "so_gubun2":"",
    "so_gubun3":"",
    "so_gubun4":"",
    "search_gubun1":"",
    "search_gubun2":"",
    "search_gubun3":"",
    "so_recom":"",
    "so_recom_Subanc":"",
    "show_sele":"10"}

contents = []

# 글 상세 페이지에 접근해서 필요한 정보 scraping
while hrefs:
    params["faq_no_n"] = re.findall(r"[\d]+", hrefs.pop(0))[1]
    
    contents.append(parse_content(contentURL, params))

In [150]:
with open("hrefs.pickle", "wb") as f:
    pickle.dump(hrefs, f)